In [1]:
import numpy as np
import pickle
import keras.backend as K

from keras.layers import  GlobalAveragePooling1D, Input, Activation, MaxPooling1D, BatchNormalization, Dense, Dropout, Conv1D,GlobalMaxPooling1D
from keras.layers import GRU,AveragePooling1D,CuDNNGRU
from keras.layers.merge import Concatenate
from keras.models import Model 
from keras.callbacks import EarlyStopping,ModelCheckpoint


import keras.backend.tensorflow_backend as KTF
import tensorflow as tf
import os


os.environ["CUDA_VISIBLE_DEVICES"] = "0"

config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)

KTF.set_session(sess)

Using TensorFlow backend.
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/xhh/anaconda3/envs/py36/lib/p

In [2]:
import numpy as np
alphabet = np.array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
                     'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'])

def label_sequence(line, MAX_SEQ_LEN, smi_ch_ind):
	X = np.zeros(MAX_SEQ_LEN)

	for i, ch in enumerate(line[:MAX_SEQ_LEN]):
		X[i] = smi_ch_ind[ch]

	return X #.tolist()

def letter_one_hot(aa):
    one_hot = np.zeros(20)
    for idx, letter in enumerate(alphabet):
        if aa == letter:
            one_hot[idx] = 1
            return one_hot


# Convert an entire protein to one-hot representation.
def protein_one_hot(protein_sequence, MAX_SEQ_LEN):
    #  Remove non-specific AA codes (very few are actually present in this dataset)
    protein_sequence = protein_sequence.replace('B', '')
    protein_sequence = protein_sequence.replace('J', '')
    protein_sequence = protein_sequence.replace('O', '')
    protein_sequence = protein_sequence.replace('U', '')
    protein_sequence = protein_sequence.replace('X', '')
    protein_sequence = protein_sequence.replace('Z', '')
    one_hot_seq = np.zeros( (MAX_SEQ_LEN, 20))
    for idx, aa in enumerate(protein_sequence[:MAX_SEQ_LEN]):
        one_hot_seq[idx, :] = letter_one_hot(aa)
    return one_hot_seq


In [3]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack
seed = 777
np.random.seed(seed)
torch.manual_seed(seed)

In [4]:
def gen_embedding(words4sent, max_seq_len, feature_dim,   to_reverse=0):
    length = []
    output = []
    
    for words in words4sent:
        if to_reverse:
            words = np.flip(words, 0)
        length.append( words.shape[0])
        if  words.shape[0] < max_seq_len:
            wordList = np.concatenate([words,np.zeros([max_seq_len - words.shape[0],feature_dim])])
        output.append(wordList)
    return np.array(output),np.array(length) 

In [5]:
def mean_pool(x, lengths):
    out = torch.FloatTensor(x.size(1), x.size(2)).zero_() # BxF
    for i in range(x.size(1)):
        out[i] = torch.mean(x[:lengths[i],i,:], 0)
    return out


class RandLSTM(nn.Module):

    def __init__(self, input_dim, num_layers, output_dim,  bidirectional=False):
        super(RandLSTM, self).__init__()
        

        self.bidirectional = bidirectional
        self.max_seq_len = 128
        self.input_dim = input_dim
         

        self.e_hid_init = torch.zeros(1, 1, output_dim)
        self.e_cell_init = torch.zeros(1, 1, output_dim)

        self.output_dim = output_dim
        self.num_layers = num_layers
        self.lm = nn.LSTM(input_dim, output_dim, num_layers=num_layers,
                          bidirectional= self.bidirectional, batch_first=True)

        self.bidirectional += 1
        
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

   

    def lstm(self, inputs, lengths):
        bsz, max_len, _ = inputs.size()
        in_embs = inputs
        lens, indices = torch.sort(lengths, 0, True)

        e_hid_init = self.e_hid_init.expand(1*self.num_layers*self.bidirectional, bsz, self.output_dim).contiguous()
        e_cell_init = self.e_cell_init.expand(1*self.num_layers*self.bidirectional, bsz, self.output_dim).contiguous()
        all_hids, (enc_last_hid, _) = self.lm(pack(in_embs[indices],
                                                        lens.tolist(), batch_first=True), (e_hid_init, e_cell_init))
        _, _indices = torch.sort(indices, 0)
        all_hids = unpack(all_hids, batch_first=True)[0][_indices]

        return all_hids

    def forward(self, words4sent):
        
        out, lengths = gen_embedding(words4sent, self.max_seq_len, self.input_dim)
        out = torch.from_numpy(out).float()
        lengths = torch.from_numpy(np.array(lengths))
        out = self.lstm(out, lengths)
#         print("output size:",out.size())
        out = out.transpose(1,0)
        out = mean_pool(out, lengths)
        return out

    def encode(self, batch):
        return self.forward(batch).cpu().detach().numpy()

In [6]:

from six.moves import cPickle as pickle #for performance

 
def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di

In [7]:
# extract w2v model
from gensim.models import Word2Vec 

w2vmodel =  Word2Vec.load('/home/xhh/PMC_model/PMC_model.txt')
def vector_name(name): 
    s = name.split(' ')
    vectors = [] 
    for word in s: 
        if w2vmodel.wv.__contains__(word):   
            vectors.append(w2vmodel.wv[word]) 
    else: 
        clear_words = re.sub('[^A-Za-z0-9]+', ' ', word) 
        clear_words = clear_words.lstrip().rstrip().split(' ')
        for w in clear_words: 
            if w2vmodel.wv.__contains__(w): 
                vectors.append(w2vmodel.wv[w]) 
    return vectors

In [8]:
# read go.obo obtain ontology type
 
obo_file = 'cross-species/go.obo'
fp=open(obo_file,'r')
obo_txt=fp.read()
fp.close()
obo_txt=obo_txt[obo_txt.find("[Term]")-1:]
obo_txt=obo_txt[:obo_txt.find("[Typedef]")]
# obo_dict=parse_obo_txt(obo_txt)
id_name_dicts = {}
for Term_txt in obo_txt.split("[Term]\n"):
    if not Term_txt.strip():
        continue
    name = ''
    ids = []
    for line in Term_txt.splitlines():
        if   line.startswith("id: "):
            ids.append(line[len("id: "):])     
        elif line.startswith("name: "):
             name=line[len("name: "):]
        elif line.startswith("alt_id: "):
            ids.append(line[len("alt_id: "):])
    
    for t_id in ids:
        id_name_dicts[t_id] = name

In [9]:
import re
protein2go =  load_dict('SC_protein2go_dicts.pkl')
prot2emb_w2v = {}
project_dim = 1024
num_layers = 1
max_go_len = 1024
max_protlen = 0
w2vlstm = RandLSTM(200,num_layers,  project_dim, bidirectional = False)


for key, value in protein2go.items(): 
    allgos = value.split(';') 
    allgos = list(set(allgos))
    count = 0
    words4sent = []
    for  go in  allgos:
        if len(go) > 2:
            feature = np.array(vector_name(id_name_dicts[go]))
            if feature.shape[0] > 0:
                words4sent.append(feature)
            
        count += feature.shape[0]
    if len(words4sent) > 0:
        sent_embedding = w2vlstm.encode(words4sent)
    else:
        sent_embedding = np.zeros((1, project_dim))
    if max_protlen < sent_embedding.shape[0]:
        max_protlen = sent_embedding.shape[0]
    prot2emb_w2v[key] = sent_embedding 

del w2vmodel

In [10]:
print(max_protlen)
# w2v_len = max_protlen
w2v_len = 32
# w2v_len = 211

24


In [11]:
prot2emb_bert = {}
max_protlen = 0
input_dim = 768
 
bertlstm = RandLSTM(input_dim,num_layers,  project_dim, bidirectional = False)
for key, value in protein2go.items():
     
    allgos = value.split(';') 
    allgos = list(set(allgos))
    count = 0
    words4sent = []
    for  go in  allgos:
        if len(go) > 2:
            feature = np.load('ncbi_allfeatures4go/'+go+'_0.npy')[1:-1]
            words4sent.append(feature)
        count += feature.shape[0] 
    if len(words4sent) > 0:
        sent_embedding = bertlstm.encode(words4sent)
    else:
        sent_embedding = np.zeros((1, project_dim))
    if max_protlen < sent_embedding.shape[0]:
        max_protlen = sent_embedding.shape[0]
    prot2emb_bert[key] = sent_embedding 

In [12]:
print(max_protlen)
# bert_len = max_protlen
bert_len = 32

24


In [14]:
import keras
feature_len = 768

max_seq_len = 1000
# max_protlen = 32
 




class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self,  ppi_pair_file, batch_size=128):
        'Initialization' 
        self.batch_size = batch_size
        self.ppi_pair_file = ppi_pair_file
        input_dim = 768
        num_layers = 1
         
        
        
        self.projection_dim = project_dim
        self.bert_len = bert_len
        self.w2v_len = w2v_len
        self.max_seqlen = max_seq_len
        self.protein2seq = load_dict('SC_protein_seqs.pkl')
        self.read_ppi()
        self.protein2onehot = {}
        self.onehot_seqs()
        self.prot2emb_bert =  prot2emb_bert
        self.prot2emb_w2v = prot2emb_w2v
         
#         self.prot2embedding() 
         
        self.on_epoch_end()
    
    def read_ppi(self):
        with open(self.ppi_pair_file, 'r') as f:
            self.ppi_pairs  =  f.readlines()
    
    def onehot_seqs(self):
        for key, value in self.protein2seq.items():
            self.protein2onehot[key] =  protein_one_hot(value, self.max_seqlen) 
    
#     def prot2embedding(self):
#         for key, value in self.protein2go.items():
#             X_go1 =  np.zeros((1,768))
#             allgos = value.split(',') 
#             allgos = list(set(allgos))
#             count = 0
#             words4sent = []
#             for  go in  allgos:
#                 if len(go) > 2:
#                     feature = np.load('../ncbi_allfeatures4go/'+go+'_0.npy')[1:-1]
#                     words4sent.append(feature)
#                 if count + feature.shape[0] > max_go_len:
#                     break    
#                 count += feature.shape[0] 
#             if len(words4sent) > 0:
#                 sent_embedding = self.bertlstm.encode(words4sent)
#             else:
#                 sent_embedding = np.zeros((1, self.projection_dim))
#             self.prot2emb[key] = sent_embedding 
    
   

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.ppi_pairs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.ppi_pairs))
         
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization

         
        X_seq1 = np.empty((self.batch_size, self.max_seqlen,20))
        X_seq2 = np.empty((self.batch_size, self.max_seqlen,20))
        y = np.empty((self.batch_size))
        X_go1_bert = np.empty((self.batch_size, self.bert_len  ,self.projection_dim))
        X_go2_bert = np.empty((self.batch_size, self.bert_len  ,self.projection_dim))
        
        X_go1_w2v = np.empty((self.batch_size, self.w2v_len  ,self.projection_dim))
        X_go2_w2v = np.empty((self.batch_size, self.w2v_len  ,self.projection_dim))


        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            ppi_pair = self.ppi_pairs[ID]
            p1, p2, label = ppi_pair.rstrip().split(',')
            if label == '1':
                y[i] = 1
            else:
                y[i] = 0
            X_seq1[i] =  self.protein2onehot[p1]
            X_seq2[i] =  self.protein2onehot[p2]
            
            prot1emb_bert = self.prot2emb_bert[p1]
            X_go1_bert[i,:prot1emb_bert.shape[0]] = prot1emb_bert
            
            prot2emb_bert = self.prot2emb_bert[p2]
            X_go2_bert[i,:prot2emb_bert.shape[0]] = prot2emb_bert
            
            prot1emb_w2v = self.prot2emb_w2v[p1]
            X_go1_w2v[i, :prot1emb_w2v.shape[0]] = prot1emb_w2v
            
            prot2emb_w2v = self.prot2emb_w2v[p2]
            X_go2_w2v[i, :  prot2emb_w2v.shape[0]  ] = prot2emb_w2v
            
             
            
            
        return [X_go1_bert, X_go2_bert, X_go1_w2v, X_go2_w2v,  X_seq1, X_seq2] ,  y



    def all_data(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization

         
        X_seq1 = np.empty((len(list_IDs_temp), self.max_seqlen,20))

         
        X_seq2 = np.empty((len(list_IDs_temp), self.max_seqlen,20))
        y = np.empty((len(list_IDs_temp)))
        X_go1_bert = np.empty((len(list_IDs_temp), self.bert_len ,self.projection_dim))
        X_go2_bert = np.empty((len(list_IDs_temp), self.bert_len  ,self.projection_dim))
    
        X_go1_w2v = np.empty((len(list_IDs_temp), self.w2v_len ,self.projection_dim))
        X_go2_w2v = np.empty((len(list_IDs_temp), self.w2v_len  ,self.projection_dim))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            ppi_pair = self.ppi_pairs[ID]
            p1, p2, label = ppi_pair.rstrip().split(',')
            if label == '1':
                y[i] = 1
            else:
                y[i] = 0
            X_seq1[i] =  self.protein2onehot[p1]
            X_seq2[i] =  self.protein2onehot[p2]
            prot1emb_bert = self.prot2emb_bert[p1]
            X_go1_bert[i,:prot1emb_bert.shape[0]] = prot1emb_bert
            
            prot2emb_bert = self.prot2emb_bert[p2]
            X_go2_bert[i,:prot2emb_bert.shape[0]] = prot2emb_bert
            
            prot1emb_w2v = self.prot2emb_w2v[p1]
            X_go1_w2v[i, :prot1emb_w2v.shape[0]] = prot1emb_w2v
            
            prot2emb_w2v = self.prot2emb_w2v[p2]
            X_go2_w2v[i, :  prot2emb_w2v.shape[0]  ] = prot2emb_w2v
            
#             prot1emb_bert = self.prot2emb_bert[p1]
#             X_go1[i,:prot1emb_bert.shape[0]] = prot1emb_bert
            
#             prot2emb_bert = self.prot2emb_bert[p2]
#             X_go2[i,:prot2emb_bert.shape[0]] = prot2emb_bert
            
#             prot1emb_w2v = self.prot2emb_w2v[p1]
#             X_go1[i,prot1emb_bert.shape[0]:prot1emb_w2v.shape[0] + prot1emb_bert.shape[0]] = prot1emb_w2v
            
#             prot2emb_w2v = self.prot2emb_w2v[p2]
#             X_go2[i,prot2emb_bert.shape[0]:prot2emb_bert.shape[0] + prot2emb_w2v.shape[0] ] = prot2emb_w2v
            
           
        return [X_go1_bert, X_go2_bert, X_go1_w2v, X_go2_w2v,  X_seq1, X_seq2] ,  y

In [15]:
from keras import backend as K, initializers, regularizers, constraints
from keras.engine.topology import Layer


def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        # todo: check that this is correct
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)


class Attention(Layer):
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True,
                 return_attention=False,
                 **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Note: The layer has been tested with Keras 1.x
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2 - Get the attention scores
            hidden = LSTM(64, return_sequences=True)(words)
            sentence, word_scores = Attention(return_attention=True)(hidden)
        """
        self.supports_masking = True
        self.return_attention = return_attention
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        eij = dot_product(x, self.W)

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        weighted_input = x * K.expand_dims(a)

        result = K.sum(weighted_input, axis=1)

        if self.return_attention:
            return [result, a]
        return result

    def compute_output_shape(self, input_shape):
        if self.return_attention:
            return [(input_shape[0], input_shape[-1]),
                    (input_shape[0], input_shape[1])]
        else:
            return input_shape[0], input_shape[-1]

In [28]:
from keras.layers import   Embedding
from keras.layers import  GRU, Bidirectional, CuDNNGRU, Lambda, Flatten
from keras.utils import multi_gpu_model
from keras.layers.merge import concatenate

def inception_block(input_tensor, output_size):
    """"""
    con1d_filters = int(output_size/4)
    y = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x1 = Conv1D(con1d_filters, 5, activation="relu", padding='same')(y)

    y = Conv1D(con1d_filters, 1, activation="relu", padding='valid')(input_tensor)
    x2 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(y)

    x3 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x4 = Conv1D(con1d_filters, 1, activation="relu", padding='same')(input_tensor)

    y = Concatenate()([x1, x2, x3, x4])
#     y = MaxPooling1D(4)(mix0)
    # y = AveragePooling1D()(mix0)
#     y = BatchNormalization()(y)

    return y

def build_cnn_gru_model(input_x, con_filters, gru_units):
    x = inception_block(input_x,con_filters )
    x = Dropout(0.3)(x)
    x_gru = Bidirectional(CuDNNGRU(gru_units, return_sequences=True))(input_x)
    x_gru = Dropout(0.3)(x_gru)
     
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_c = Attention()(x)
    x_gru_a = GlobalAveragePooling1D()(x_gru)
    x_gru_b = GlobalMaxPooling1D()(x_gru)
    x_gru_c = Attention()(x_gru)
    x = Concatenate()([ x_a, x_b, x_c, x_gru_c, x_gru_b,  x_gru_a])
    x = Dense(256)(x)
    return x


def build_cnn_model(input_x, con_filters, gru_units):
    x = inception_block(input_x,con_filters )
    x = Dropout(0.3)(x)
     
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_c = Attention()(x)
    
    x = Concatenate()([ x_a, x_b, x_c])
#     x = Dense(256)(x)
    return x

def build_model():
    con_filters = 128
    gru_units = 64
    left_input_go_bert = Input(shape=(bert_len ,project_dim))
    right_input_go_bert = Input(shape=(bert_len,project_dim))
    left_input_go_w2v = Input(shape=(w2v_len,project_dim))
    right_input_go_w2v = Input(shape=(w2v_len,project_dim))
    
    
    left_input_seq = Input(shape=(max_seq_len,20))
    right_input_seq = Input(shape=(max_seq_len,20))
    
    

    left_x_go_bert = build_cnn_model(left_input_go_bert, con_filters, gru_units)
    right_x_go_bert = build_cnn_model(right_input_go_bert, con_filters,gru_units)
    left_x_seq = build_cnn_model(left_input_seq, con_filters, gru_units)
    right_x_seq = build_cnn_model(right_input_seq, con_filters, gru_units)
    
    left_x_go_w2v = build_cnn_model(left_input_go_w2v, con_filters, gru_units)
    right_x_go_w2v = build_cnn_model(right_input_go_w2v, con_filters,gru_units)
    
      
    left_x =   Concatenate()([left_x_go_bert  ,   left_x_go_w2v,   left_x_seq ])
    left_x = Dense(256, activation='relu')(left_x)
    
    
    right_x =   Concatenate()([right_x_go_bert,  right_x_go_w2v,  right_x_seq])
    right_x = Dense(256, activation='relu')(right_x)
    
    
    
    x = Concatenate()([left_x, right_x])
#     x =   Concatenate()([left_x_go_bert  , right_x_go_bert, left_x_go_w2v, right_x_go_w2v, left_x_seq, right_x_seq])
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.1)(x)
    
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(512, activation='relu')(x)
  
     
    x = Dense(1)(x)
    output = Activation('sigmoid')(x)
    # model = Model([left_input_go, right_input_go], output)
  
    model = Model([left_input_go_bert, right_input_go_bert, left_input_go_w2v , right_input_go_w2v,   left_input_seq, right_input_seq], output)
#     model = multi_gpu_model(model, gpus=2)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = build_model()
print(model.summary())

Model: "model_19"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_109 (InputLayer)          (None, 32, 1024)     0                                            
__________________________________________________________________________________________________
input_111 (InputLayer)          (None, 32, 1024)     0                                            
__________________________________________________________________________________________________
input_113 (InputLayer)          (None, 1000, 20)     0                                            
__________________________________________________________________________________________________
input_110 (InputLayer)          (None, 32, 1024)     0                                            
___________________________________________________________________________________________

In [ ]:
from sklearn.model_selection import StratifiedKFold
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from keras.utils import multi_gpu_model

dataset_name = 'sc'
for rep in range(5):
    n_splits = 1
    TPRs =  np.zeros(n_splits)
    FPRs = np.zeros(n_splits)
    Precs = np.zeros(n_splits)
    ACCs = np.zeros(n_splits)
    F1s = np.zeros(n_splits)
    MCCs = np.zeros(n_splits)
    AUCs = np.zeros(n_splits)
     
    count = 0
    for split in range(n_splits):
        train_pairs_file = 'SC_CV/train'+str(rep)+'-'+str(split)
        test_pairs_file = 'SC_CV/test'+str(rep)+'-'+str(split)
        valid_pairs_file = 'SC_CV/valid'+str(rep)+'-'+str(split)

        batch_size = 128
        train_generator = DataGenerator(   train_pairs_file,batch_size = batch_size )
        test_generator = DataGenerator(   test_pairs_file,batch_size = batch_size)
        valid_generator = DataGenerator(   valid_pairs_file,batch_size = batch_size)
         
        # model = build_model_without_att()
        model = build_model()
        save_model_name = 'SC_CV/fusion_sent_GoplusSeq'+str(rep)+'-'+str(split) + '.hdf5'
        
        earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
        save_checkpoint = ModelCheckpoint(save_model_name, save_best_only=True, monitor='val_loss', mode='min', save_weights_only=True)

         
        # validation_data = (valid_X, valid_Y),  verbose=1,callbacks=[earlyStopping, save_checkpoint]
        #  max_queue_size=16, workers=8, use_multiprocessing=True,
        # validation_data=valid_generator,callbacks=[earlyStopping, save_checkpoint] 
        hist = model.fit_generator(generator=train_generator,
                    epochs = 100,verbose=1,validation_data=valid_generator,callbacks=[earlyStopping, save_checkpoint])
         
        
        # model = load_model(save_model_name)
        model.load_weights(save_model_name)
        with open(test_pairs_file, 'r') as f:
            test_ppi_pairs  =  f.readlines()

        test_len = len(test_ppi_pairs) 
        list_IDs_temp = np.arange(test_len)

        test_x, y_test = test_generator.all_data(list_IDs_temp)

        y_pred_prob = model.predict(test_x)

       
        y_pred = (y_pred_prob > 0.5)
        auc = metrics.roc_auc_score(y_test, y_pred_prob) 
        f1 = f1_score(y_test, y_pred)
        pre = precision_score(y_test, y_pred)
        acc = accuracy_score(y_test, y_pred)

        precision, recall, _thresholds = metrics.precision_recall_curve(y_test, y_pred_prob)
        pr_auc = metrics.auc(recall, precision)
        mcc = matthews_corrcoef(y_test, y_pred)

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        total=tn+fp+fn+tp
        sen = float(tp)/float(tp+fn)
        sps = float(tn)/float((tn+fp))

        tpr = float(tp)/float(tp+fn)
        fpr = float(fp)/float((tn+fp))
        print('--------------------------\n')
        print ('AUC: %f' % auc)
        print ('ACC: %f' % acc) 
        # print("PRAUC: %f" % pr_auc)
        print ('MCC : %f' % mcc)
        # print ('SEN: %f' % sen)
        # print ('SEP: %f' % sps)
        print('TPR:%f'%tpr)
        print('FPR:%f'%fpr)
        print('Pre:%f'%pre)
        print('F1:%f'%f1)
        print('--------------------------\n')
        TPRs[count] = tpr
        FPRs[count] = fpr
        Precs[count] =pre
        ACCs[count] =acc
        F1s[count] =f1
        MCCs[count] =mcc
        AUCs[count] =auc
        count += 1
        del test_x
        del y_test
    print ('mean AUC: %f' % np.mean(AUCs))
    print ('mean ACC: %f' % np.mean(ACCs)) 
    print ('mean MCC : %f' % np.mean(MCCs))
    print('mean TPR:%f'% np.mean(TPRs))
    print('mean FPR:%f'% np.mean(FPRs))
    print('mean Pre:%f'% np.mean(Precs))
    print('mean F1:%f'% np.mean(F1s))
    np.savez('fusion_sent_seq_and_go__incep_'+str(rep), AUCs=AUCs, ACCs=ACCs, MCCs=MCCs, TPRs = TPRs, FPRs=FPRs, Precs=Precs, F1s=F1s)
    
    
    
    
    

Epoch 1/100
135/135 [==============================] - 50s 370ms/step - loss: 0.4887 - acc: 0.7517 - val_loss: 0.3757 - val_acc: 0.8143
Epoch 2/100
135/135 [==============================] - 28s 205ms/step - loss: 0.4561 - acc: 0.8145 - val_loss: 0.3439 - val_acc: 0.8485
Epoch 3/100
135/135 [==============================] - 29s 211ms/step - loss: 0.3250 - acc: 0.8594 - val_loss: 0.3321 - val_acc: 0.8492
Epoch 4/100
135/135 [==============================] - 27s 202ms/step - loss: 0.2510 - acc: 0.8979 - val_loss: 0.3103 - val_acc: 0.8642
Epoch 5/100
135/135 [==============================] - 30s 222ms/step - loss: 0.1925 - acc: 0.9262 - val_loss: 0.2739 - val_acc: 0.9009
Epoch 6/100
135/135 [==============================] - 27s 201ms/step - loss: 0.1696 - acc: 0.9383 - val_loss: 0.1751 - val_acc: 0.9310
Epoch 7/100
135/135 [==============================] - 28s 207ms/step - loss: 0.2110 - acc: 0.9163 - val_loss: 0.1927 - val_acc: 0.9280
Epoch 8/100
135/135 [===========================

Epoch 1/100
135/135 [==============================] - 60s 441ms/step - loss: 0.5990 - acc: 0.6661 - val_loss: 0.4617 - val_acc: 0.7879
Epoch 2/100
135/135 [==============================] - 28s 208ms/step - loss: 0.3933 - acc: 0.8222 - val_loss: 0.3945 - val_acc: 0.8359
Epoch 3/100
135/135 [==============================] - 29s 216ms/step - loss: 0.3088 - acc: 0.8729 - val_loss: 0.2794 - val_acc: 0.8945
Epoch 4/100
135/135 [==============================] - 29s 212ms/step - loss: 0.2332 - acc: 0.9072 - val_loss: 0.3316 - val_acc: 0.8580
Epoch 5/100
135/135 [==============================] - 29s 216ms/step - loss: 0.2453 - acc: 0.9035 - val_loss: 0.2001 - val_acc: 0.9324
Epoch 6/100
135/135 [==============================] - 27s 202ms/step - loss: 0.1725 - acc: 0.9348 - val_loss: 0.2167 - val_acc: 0.9110
Epoch 7/100
135/135 [==============================] - 30s 219ms/step - loss: 0.1634 - acc: 0.9395 - val_loss: 0.1527 - val_acc: 0.9454
Epoch 8/100
135/135 [===========================

Epoch 17/100
135/135 [==============================] - 22s 165ms/step - loss: 0.1290 - acc: 0.9516 - val_loss: 0.1534 - val_acc: 0.9412
Epoch 18/100
135/135 [==============================] - 23s 170ms/step - loss: 0.1049 - acc: 0.9608 - val_loss: 0.1517 - val_acc: 0.9433
Epoch 19/100
135/135 [==============================] - 22s 162ms/step - loss: 0.0911 - acc: 0.9671 - val_loss: 0.1298 - val_acc: 0.9536
Epoch 20/100
135/135 [==============================] - 22s 166ms/step - loss: 0.0904 - acc: 0.9678 - val_loss: 0.1317 - val_acc: 0.9516
Epoch 21/100
135/135 [==============================] - 23s 167ms/step - loss: 0.0852 - acc: 0.9692 - val_loss: 0.1295 - val_acc: 0.9525
Epoch 22/100
135/135 [==============================] - 23s 168ms/step - loss: 0.1069 - acc: 0.9613 - val_loss: 0.1350 - val_acc: 0.9510
Epoch 23/100
135/135 [==============================] - 24s 175ms/step - loss: 0.0875 - acc: 0.9694 - val_loss: 0.1417 - val_acc: 0.9467
Epoch 24/100
135/135 [===================

In [27]:
import numpy as np 
results1 =   np.load( 'fusion_sent_seq_and_go__incep_0.npz')
results2 =   np.load( 'fusion_sent_seq_and_go__incep_1.npz')
results3 =   np.load( 'fusion_sent_seq_and_go__incep_2.npz')
results4 =   np.load( 'fusion_sent_seq_and_go__incep_3.npz')
results5 =   np.load( 'fusion_sent_seq_and_go__incep_4.npz')
  

print ('mean AUC: %f' %  (  (np.mean( results4[ 'AUCs' ] )  +  np.mean( results5[ 'AUCs' ] )  + np.mean( results1[ 'AUCs' ] )  + np.mean(  results2[ 'AUCs' ] )  + np.mean(results3[ 'AUCs' ]))/5     ) )
print ('mean ACC: %f' %   (  ( np.mean( results4[ 'ACCs' ] )  + np.mean(  results5[ 'ACCs' ] )  +   np.mean( results1[ 'ACCs' ] )  + np.mean(  results2[ 'ACCs' ] )  + np.mean(results3[ 'ACCs' ]))/5) )
print ('mean MCC : %f' %  ( ( np.mean( results4[ 'MCCs' ] )  + np.mean(  results5[ 'MCCs' ] )  + np.mean( results1[ 'MCCs' ] )  + np.mean(  results2[ 'MCCs' ] )  + np.mean(results3[ 'MCCs' ])     )/5))
print('mean TPR:%f'%    (( np.mean( results4[ 'TPRs' ] )  + np.mean(  results5[ 'TPRs' ] )  + np.mean( results1[ 'TPRs' ] )  + np.mean(  results2[ 'TPRs' ] )  + np.mean(results3[ 'TPRs' ])     )/5))
print('mean FPR:%f'%   (  (np.mean( results4[ 'FPRs' ] )  + np.mean(  results5[ 'FPRs' ] )  + np.mean( results1[ 'FPRs' ] )  + np.mean(  results2[ 'FPRs' ] )  + np.mean(results3[ 'FPRs' ])     )/5))
print('mean Pre:%f'%    ( (np.mean( results4[ 'Precs' ] )  + np.mean(  results5[ 'Precs' ] )  + np.mean( results1[ 'Precs' ] )  + np.mean(  results2[ 'Precs' ] )  + np.mean(results3[ 'Precs' ])     )/5))
print('mean F1:%f'%    (  (np.mean( results4[ 'F1s' ] )  + np.mean(  results5[ 'F1s' ] )  +np.mean( results1[ 'F1s' ] )  + np.mean(  results2[ 'F1s' ] )  + np.mean(results3[ 'F1s' ])     )/5))

mean AUC: 0.987056
mean ACC: 0.955169
mean MCC : 0.911190
mean TPR:0.934556
mean FPR:0.024108
mean Pre:0.975085
mean F1:0.954354


In [19]:
import numpy as np 
results1 =   np.load( 'fusion_sent_seq_and_go__incep_0.npz')
results2 =   np.load( 'fusion_sent_seq_and_go__incep_1.npz')
results3 =   np.load( 'fusion_sent_seq_and_go__incep_2.npz')
results4 =   np.load( 'fusion_sent_seq_and_go__incep_3.npz')
results5 =   np.load( 'fusion_sent_seq_and_go__incep_4.npz')
  

print ('mean AUC: %f' %  (  (np.std( np.concatenate( [results4[ 'AUCs' ], results3[ 'AUCs' ], results2[ 'AUCs' ],results1[ 'AUCs' ],results5[ 'AUCs' ]   ]  )     ))     ) )
print ('mean ACC: %f' %   (  ( np.std(np.concatenate( [results4[ 'ACCs' ], results3[ 'ACCs' ],results2[ 'ACCs' ],results1[ 'ACCs' ],results5[ 'ACCs' ] ]  ) ))) )
print ('mean MCC : %f' %  ( ( np.std(  np.concatenate( [results4[ 'MCCs' ],results3[ 'MCCs' ],results2[ 'MCCs' ],results1[ 'MCCs' ],results5[ 'MCCs' ] ])  ))))
print('mean TPR:%f'%    (( np.std( np.concatenate( [ results4[ 'TPRs' ],results3[ 'TPRs' ],results2[ 'TPRs' ],results1[ 'TPRs' ], results5[ 'TPRs' ]]) ))))
print('mean FPR:%f'%   (  (np.std( np.concatenate( [ results4[ 'FPRs' ],results3[ 'FPRs' ],results2[ 'FPRs' ],results1[ 'FPRs' ], results5[ 'FPRs' ]])  )     )))
print('mean Pre:%f'%    ( (np.std( np.concatenate( [ results4[ 'Precs' ],results3[ 'Precs' ],results2[ 'Precs' ],results1[ 'Precs' ],results5[ 'Precs' ]])   )     )))
print('mean F1:%f'%    (  (np.std( np.concatenate( [results4[ 'F1s' ],results3[ 'F1s' ],results2[ 'F1s' ],results1[ 'F1s' ],results5[ 'F1s' ] ])      )     )))
      
      
      

mean AUC: 0.000600
mean ACC: 0.000921
mean MCC : 0.001651
mean TPR:0.005312
mean FPR:0.004405
mean Pre:0.004516
mean F1:0.000909
